In [1]:
import pathlib
import torch

from esm import FastaBatchedDataset, pretrained

In [2]:
import os
import pandas as pd

In [3]:
import requests

def fetch_protein_data(base_term, variants=['','a','b','c','d'], topn=200):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    # Constructing the search term with explicit variants
    all_results = []
    all_search = []
    for variant in variants:
        term = f"{base_term}{variant}"

        params = {
            'db': 'protein',
            'term': f"{term}[title]",  # Searches only in the titles
            'retmax': topn,
            'retmode': 'json'
        }

        response = requests.get(base_url, params=params)
        print(f"Status Code: {response.status_code}")
        if response.status_code == 200:
            search_data = response.json()
            all_results.extend(search_data['esearchresult']['idlist'])
            all_search.append(search_data)
    return all_results,all_search
# jj,qq  =fetch_protein_data('cas1', variants=['','a','b','c','d'], topn=200)

In [4]:
from Bio import Entrez
from Bio import SeqIO

def download_protein_sequences(protein_ids, email, output_file='protein_sequences.fasta'):
    # Set the email address to be used by NCBI for usage monitoring
    Entrez.email = email
    
    # Open the output file in write mode
    with open(output_file, 'w') as out_file:
        # Process each protein ID
        for protein_id in protein_ids:
            # Fetch the sequence from NCBI
            handle = Entrez.efetch(db='protein', id=protein_id, rettype='fasta', retmode='text')
            # Read the sequence from the handle
            record = SeqIO.read(handle, 'fasta')
            # Write the sequence to the output file
            SeqIO.write(record, out_file, 'fasta')
            # Close the handle
            handle.close()
            
    print(f"Sequences have been saved to {output_file}")

# Example usage
# protein_ids = ['YP_009724390', 'NP_000240']  # Add your list of NCBI protein IDs here
email = 'your.email@example.com'  # Replace with your email
# download_protein_sequences(xxx,  'sam.salari@roche.com','/home/salaris/esm_atlas/data/cas1/cas1.fasta')


In [5]:
caslist = ['cas1','cas2','cas3','cas4','cas5','cas6','cas7','cas8','cas9','cas10','cas11','cas12','cas13']
# caslist = ['cas9']

In [6]:
all_cas_ids = dict()
for cas in caslist:
    xx,_ = fetch_protein_data(cas,variants=['','a','b','c','d'],topn= 5000)
    all_cas_ids.update({cas:xx})

Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 2

In [7]:
for cas in caslist:
    print(cas, len(all_cas_ids[cas]))

cas1 14675
cas2 5000
cas3 5001
cas4 5173
cas5 13656
cas6 5019
cas7 3236
cas8 265
cas9 5006
cas10 110
cas11 86
cas12 149
cas13 775


In [8]:


for cas in caslist:
    
    casfolder = f"/home/salaris/protein_model/data/{cas}/"
    print(casfolder)
    os.makedirs(os.path.dirname(casfolder), exist_ok=True)
    
    xx = all_cas_ids[cas]
    casfasta = f'{casfolder}/{cas}_sequence.fasta'
    print(casfasta)
    download_protein_sequences(protein_ids=xx,email=  'sam.salari@roche.com',output_file=casfasta)


/home/salaris/protein_model/data/cas1/
/home/salaris/protein_model/data/cas1//cas1_sequence.fasta
Sequences have been saved to /home/salaris/protein_model/data/cas1//cas1_sequence.fasta
/home/salaris/protein_model/data/cas2/
/home/salaris/protein_model/data/cas2//cas2_sequence.fasta
Sequences have been saved to /home/salaris/protein_model/data/cas2//cas2_sequence.fasta
/home/salaris/protein_model/data/cas3/
/home/salaris/protein_model/data/cas3//cas3_sequence.fasta
Sequences have been saved to /home/salaris/protein_model/data/cas3//cas3_sequence.fasta
/home/salaris/protein_model/data/cas4/
/home/salaris/protein_model/data/cas4//cas4_sequence.fasta
Sequences have been saved to /home/salaris/protein_model/data/cas4//cas4_sequence.fasta
/home/salaris/protein_model/data/cas5/
/home/salaris/protein_model/data/cas5//cas5_sequence.fasta
Sequences have been saved to /home/salaris/protein_model/data/cas5//cas5_sequence.fasta
/home/salaris/protein_model/data/cas6/
/home/salaris/protein_model/dat

In [9]:
! ls /home/salaris/protein_model/data/

all_data_20240628_17.csv  cas10  cas12	cas2  cas4  cas6  cas8
cas1			  cas11  cas13	cas3  cas5  cas7  cas9


In [10]:
## build the training and test set:

In [11]:
data_list = []
for cas in caslist:
    
    casfolder = f"/home/salaris/protein_model/data/{cas}/"
    print(casfolder)
  
    casfasta = f'{casfolder}/{cas}_sequence.fasta'
    # read the cas fasta file and convert it into a dataframe
    for record in SeqIO.parse(casfasta, "fasta"):
        data_list.append({"seq": str(record.seq),
                         "description": str(record.description),
                         "record_id": str(record.id),
                         "record_name": str(record.name),
                         "class": cas})
        

/home/salaris/protein_model/data/cas1/
/home/salaris/protein_model/data/cas2/
/home/salaris/protein_model/data/cas3/
/home/salaris/protein_model/data/cas4/
/home/salaris/protein_model/data/cas5/
/home/salaris/protein_model/data/cas6/
/home/salaris/protein_model/data/cas7/
/home/salaris/protein_model/data/cas8/
/home/salaris/protein_model/data/cas9/
/home/salaris/protein_model/data/cas10/
/home/salaris/protein_model/data/cas11/
/home/salaris/protein_model/data/cas12/
/home/salaris/protein_model/data/cas13/


In [12]:
all_data_df = pd.DataFrame(data_list)

In [13]:
import datetime

# Get the current date and time
current_datetime = datetime.datetime.now()

# Format the date string
date_string = current_datetime.strftime('%Y%m%d_%H')

print(date_string)
all_data_filename = f'/home/salaris/protein_model/data/all_data_{date_string}.csv'

20240629_09


In [21]:
all_data_df.to_csv(all_data_filename,sep ='\t')

In [22]:
all_data_df


,Unnamed: 0,seq,description,record_id,record_name,class
0,0,MERIVDIATDGRHLSAHRGFMIVSAERQEIGRIPLDDVAAVVVHAH...,XBY42976.1 type II CRISPR-associated endonucle...,XBY42976.1,XBY42976.1,cas1
1,1,MGWRSIIIGQHAKITHSAHMMVVQTKDGINEIPMDDIAKVLVETTQ...,WP_350341823.1 type II CRISPR-associated endon...,WP_350341823.1,WP_350341823.1,cas1
2,2,MGWRNVIITQHSKLSYSSNMMVVHTKDGVSQIPVSDINMLVVLTTQ...,XBY00350.1 type II CRISPR-associated endonucle...,XBY00350.1,XBY00350.1,cas1
3,3,MGWRSIIIGQHAKITHSAHMMVVQTKDGINEIPMDDIAKVLVETTQ...,XBX92904.1 type II CRISPR-associated endonucle...,XBX92904.1,XBX92904.1,cas1
4,4,MGWRSVIITQHAKLTYSMNMMIVQTKDGINQIPISDINLLLVSTSQ...,XBX95754.1 type II CRISPR-associated endonucle...,XBX95754.1,XBX95754.1,cas1
...,...,...,...,...,...,...
58146,58146,MAKKNKMKPRELREAQKKARQLKAAEINNNAIPAIAAMPAAEAAAP...,WP_118125476.1 type VI-D CRISPR-associated RNA...,WP_118125476.1,WP_118125476.1,cas13
58147,58147,MAKKNKMKPRELREAQKKARQLKAAEINNNAAPAIAAMPAAEVIAP...,WP_117897534.1 MULTISPECIES: type VI-D CRISPR-...,WP_117897534.1,WP_117897534.1,cas13
58148,58148,MAKKSKGMSLREKRELEKQKRIQKAAVNSVNDTPEKTEEANVVSVN...,WP_074833651.1 type VI-D CRISPR-associated RNA...,WP_074833651.1,WP_074833651.1,cas13
58149,58149,MAKKNKMKPRELREAQKKARQLKAAEINNNAVPAIAAMPAAEAAAP...,WP_041337480.1 type VI-D CRISPR-associated RNA...,WP_041337480.1,WP_041337480.1,cas13


## Split the dataset into train and validation set:

In [23]:
all_data_df = pd.read_csv(all_data_filename,sep = '\t')
all_data_df.head()

,Unnamed: 0.1,Unnamed: 0,seq,description,record_id,record_name,class
0,0,0,MERIVDIATDGRHLSAHRGFMIVSAERQEIGRIPLDDVAAVVVHAH...,XBY42976.1 type II CRISPR-associated endonucle...,XBY42976.1,XBY42976.1,cas1
1,1,1,MGWRSIIIGQHAKITHSAHMMVVQTKDGINEIPMDDIAKVLVETTQ...,WP_350341823.1 type II CRISPR-associated endon...,WP_350341823.1,WP_350341823.1,cas1
2,2,2,MGWRNVIITQHSKLSYSSNMMVVHTKDGVSQIPVSDINMLVVLTTQ...,XBY00350.1 type II CRISPR-associated endonucle...,XBY00350.1,XBY00350.1,cas1
3,3,3,MGWRSIIIGQHAKITHSAHMMVVQTKDGINEIPMDDIAKVLVETTQ...,XBX92904.1 type II CRISPR-associated endonucle...,XBX92904.1,XBX92904.1,cas1
4,4,4,MGWRSVIITQHAKLTYSMNMMIVQTKDGINQIPISDINLLLVSTSQ...,XBX95754.1 type II CRISPR-associated endonucle...,XBX95754.1,XBX95754.1,cas1


In [55]:
import pandas as pd
from sklearn.model_selection import train_test_split


# Splitting the DataFrame into train and test sets
target = all_data_df['class']
train, validation = train_test_split(all_data_df, test_size=0.1, random_state=42,stratify=target)

# train and test are now DataFrames containing the split data


In [56]:
train.shape,validation.shape

((52335, 7), (5816, 7))

In [46]:
# Use train for trainig and testing , 
# use validation data for final validation 

train_filename = all_data_filename + 'train_test.csv'
train.to_csv(train_filename, sep = '\t')

validation_filename = all_data_filename + 'tfinal_validation.csv'
validation.to_csv(validation_filename, sep = '\t')

In [47]:
(train_filename, validation_filename)

('/home/salaris/protein_model/data/all_data_20240629_09.csvtrain_test.csv',
 '/home/salaris/protein_model/data/all_data_20240629_09.csvtfinal_validation.csv')

In [60]:
train


,Unnamed: 0.1,Unnamed: 0,seq,description,record_id,record_name,class
50708,50708,50708,MTLTSSSTLPFEDALRPGSPLLDPGKRHDFVLVFQVKDGNPNGDPD...,WP_110885806.1 type I CRISPR-associated protei...,WP_110885806.1,WP_110885806.1,cas7
16924,16924,16924,MSQYTRYLVCYDIQDSKVRRRFFSFLKDLGLVPLQESVFYGDLKPA...,WP_337559137.1 CRISPR-associated endonuclease ...,WP_337559137.1,WP_337559137.1,cas2
19892,19892,19892,MAGKTDPENTSLWLPLWMHLWDTAGIMERLVRQWLPESAKRAMGFE...,WP_348816664.1 CRISPR-associated helicase Cas3...,WP_348816664.1,WP_348816664.1,cas3
24855,24855,24855,MTFGELRMLRRGKLRMPIRRIRTHVLPGIDWILVGLTAALCGAGAL...,WP_322779726.1 rod shape-determining protein R...,WP_322779726.1,WP_322779726.1,cas4
43272,43272,43272,MRNQIEFKLYGKYALFSDPVTRVGGEKFSYQIPTYQALKGICESIY...,SKB36650.1 CRISPR-associated protein Cas5d [Ac...,SKB36650.1,SKB36650.1,cas5
...,...,...,...,...,...,...,...
1206,1206,1206,MIKRTLFFGNPAYLSTQNDQLILKYPDSVEEKTIPIEDLGFVVLEH...,WP_339624695.1 type II CRISPR-associated endon...,WP_339624695.1,WP_339624695.1,cas1
13384,13384,13384,MRQFLNTLFVLNESAYLSLDGENVVVSCDKAEVGRVALHTLESILC...,WP_097801864.1 type I-C CRISPR-associated endo...,WP_097801864.1,WP_097801864.1,cas1
2477,2477,2477,MIKVLLNTLYVQTQQSYIRLDHETIVVEVEGAKTLQVPIHHIGAIV...,WP_328075407.1 CRISPR-associated endonuclease ...,WP_328075407.1,WP_328075407.1,cas1
8954,8954,8954,MKKSYYLFNPGRMSREDNTLKFAVTDDNGNESSTKYIPVETVDNLY...,UPK72077.1 type I-B CRISPR-associated endonucl...,UPK72077.1,UPK72077.1,cas1


In [62]:
#this function will split the original fasta into train or validation fasta files that can be later read using the esms fasta2dataset function 
from Bio import SeqIO

def split_fasta_by_ids(fasta_file, train_ids, test_ids, train_outfile, test_outfile):
    """
    Splits a FASTA file into training and validation files based on provided lists of IDs.
    
    Parameters:
    - fasta_file: Path to the input FASTA file.
    - train_ids: List of IDs for the training set.
    - test_ids: List of IDs for the test set.
    - train_outfile: Path to the output FASTA file for the training set.
    - test_outfile: Path to the output FASTA file for the validation set.
    """
    # Read the fasta file
    records = list(SeqIO.parse(fasta_file, 'fasta'))
    
    # Separate records based on IDs
    train_records = [record for record in records if record.id in train_ids]
    test_records = [record for record in records if record.id in test_ids]
    
    # Write the records to separate fasta files
    SeqIO.write(train_records, train_outfile, 'fasta')
    SeqIO.write(test_records, test_outfile, 'fasta')

# # Example usage
# train_ids = ['id1', 'id3']  # Example training IDs
# test_ids = ['id2', 'id4']   # Example test IDs

# split_fasta_by_ids('path/to/your/input.fasta', train_ids, test_ids, 'training.fasta', 'validation.fasta')


In [76]:
for cas in caslist:
    casfolder = f"/home/salaris/protein_model/data/{cas}/"
    casfasta = f'{casfolder}/{cas}_sequence.fasta'
    training_fasta_file = casfolder + cas + '_training.fasta'
    validation_fasta_file = casfolder + cas + '_validation.fasta'
    train_ids = train.record_id.to_list()
    validation_ids = validation.record_id.to_list()
    split_fasta_by_ids(casfasta, train_ids, validation_ids, training_fasta_file, validation_fasta_file)
